In [1]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


In [7]:
data_pose = pd.read_csv("./csv_files/11_poses_data_pose.csv")
features = data_pose.drop(["pose"], axis=1)
target = data_pose[["pose"]]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.6)

data_all_pose_model = RandomForestClassifier()
data_all_pose_model.fit(X_train, y_train)

print(classification_report(y_test, data_all_pose_model.predict(X_test)))


              precision    recall  f1-score   support

           0       0.92      0.97      0.94        34
           1       0.99      0.99      0.99       116
           2       0.91      0.96      0.93       110
           3       1.00      0.80      0.89        30
           4       1.00      1.00      1.00        41
           5       0.89      0.98      0.93       123
           6       1.00      0.31      0.47        13
           7       0.96      0.97      0.97       118
           8       1.00      0.93      0.96        41
           9       0.96      0.96      0.96        45
          10       0.97      0.93      0.95       118

    accuracy                           0.95       789
   macro avg       0.96      0.89      0.91       789
weighted avg       0.95      0.95      0.95       789



In [ ]:
names = {
        0: "Ardha Chandrasana",
        1: "Downdog",
        2: "Goddess",
        3: "Marjaryasana",
        4: "Padmasana",
        5: "Plank",
        6: "Sivasana",
        7: "Tree",
        8: "Urdhvamukha shvanasana",
        9: "Utkatasana",
        10: "Warrior"
    }


In [8]:
pk.dump(data_all_pose_model, open("./models/11_poses.model", "wb"))


In [29]:
poses = {
    0: "Anantasana",
    1: "Ardhakati_Chakrasana",
    2: "Bhujangasana",
    3: "Kati_Chakrasana",
    4: "Marjariasana",
    5: "Parvatasana",
    6: "Sarvangasana",
    7: "Tadasana",
    8: "Vajrasana",
    9: "Viparita_Karani"
}


In [30]:
data_pose = pd.read_csv("./csv_files/data_pose.csv")
features = data_pose.drop(["pose"], axis=1)
target = data_pose[["pose"]]


In [32]:
loaded_model = pk.load(open("./models/data_pose.model", "rb"))
result = loaded_model.score(features, target)
print(result)


1.0
